In [1]:
import os
import numpy as np
import faiss
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_groq import ChatGroq 
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


import os
from dotenv import load_dotenv

load_dotenv()
groq_api_key = os.getenv("groq_api")



# --- 2. Initialize Models and Components ---

# LLM for generating hypothetical documents (e.g., OpenAI's gpt-3.5-turbo)
# You can use other LLMs like Groq's ChatGroq if you prefer, with appropriate imports
llm = ChatGroq(
        model_name='gemma2-9b-it',
        temperature=0, # Keep temperature at 0 for more factual/less creative answers
        groq_api_key=groq_api_key
    )

# Embedding model for both hypothetical documents and real documents
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
print("Embedding model (all-MiniLM-L6-v2) initialized.")




C:\Users\nis44\AppData\Local\Temp\ipykernel_10432\3211662931.py:30: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\nis44\anaconda3\RAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\nis44\anaconda3\RAG\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication,

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


W0616 16:51:27.460000 10432 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Embedding model (all-MiniLM-L6-v2) initialized.


In [2]:
# --- 3. Create a Dummy Vector Store (If you don't have one saved) ---
# In a real scenario, your 'docs' would come from loaded data and your 'vectorstore'
# would be built from them. For this example, we'll create a small one on the fly.

# Sample content for our "knowledge base"
raw_documents = [
    "Reinforcement Learning from Human Feedback (RLHF) is a technique that fine-tunes language models.",
    "It uses human preferences to train a reward model, which then guides the LLM.",
    "The primary goal of RLHF is to align AI behavior with human values, making models more helpful and harmless.",
    "Without RLHF, LLMs might generate undesirable outputs like toxic or biased content.",
    "HyDE stands for Hypothetical Document Embedding, and it improves retrieval by generating a sample document.",
    "Step-Back Prompting enhances LLM reasoning by making the model derive high-level concepts first.",
    "A Reflection Agent allows an LLM to self-critique and refine its own answers iteratively.",
    "FAISS is a library for efficient similarity search and clustering of dense vectors.",
    "LangChain provides frameworks for building applications with LLMs, including RAG and agents.",
    "Groq offers very fast inference for LLMs like Llama3."
]

# Create embeddings for our dummy documents and build a FAISS index in memory
print("\nCreating a dummy FAISS vector store with sample documents...")
# Ensure docs are processed into LangChain Document objects if not already
from langchain_core.documents import Document
documents_for_faiss = [Document(page_content=d) for d in raw_documents]
vectorstore = FAISS.from_documents(documents_for_faiss, embedding_model)
docs = [doc.page_content for doc in documents_for_faiss] # Original texts list
print(f"Dummy FAISS vector store created with {len(docs)} documents.")


Creating a dummy FAISS vector store with sample documents...
Dummy FAISS vector store created with 10 documents.


In [3]:
# --- 4. Define the HyDE Prompt Template ---
# This prompt instructs the LLM to generate a hypothetical document.
hyde_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Please write a short, relevant, and well-written hypothetical document "
     "that could answer the user's question. This document should be detailed "
     "and sound like a real piece of text from a knowledge base.\n\n"
     "Hypothetical Document:"), # We want the LLM to fill this
    ("user", "{question}")
])
print("HyDE Prompt Template defined.")


HyDE Prompt Template defined.


In [ ]:
# --- 5. Create the HyDE Chain ---
# This chain takes a question, generates a hypothetical document, and outputs it as a string.
hyde_chain = hyde_prompt | llm | StrOutputParser()
print("HyDE Chain (LLM for generation) defined.")


HyDE Chain (LLM for generation) defined.


In [5]:
# --- 6. The HyDE Retrieval Function ---
def hyde_retrieve(query: str, k: int = 3) -> list[str]:
    """
    Performs retrieval using the Hypothetical Document Embedding (HyDE) technique.

    Args:
        query (str): The user's original short query.
        k (int): The number of top relevant real documents to retrieve.

    Returns:
        list[str]: A list of the actual retrieved document texts.
    """
    print(f"\n--- HyDE Retrieval for query: '{query}' ---")
    
    # Step A: Generate the hypothetical document
    print("Generating hypothetical document...")
    hypothetical_document = hyde_chain.invoke({"question": query})
    print(f"Hypothetical Document Generated:\n---\n{hypothetical_document[:200]}...\n---")

    # Step B: Embed the hypothetical document
    print("Embedding hypothetical document...")
    # embed_documents expects a list, even if it's just one document
    hypothetical_embedding = embedding_model.embed_documents([hypothetical_document])[0] # [0] to get the single vector
    hypothetical_embedding_np = np.array(hypothetical_embedding).astype('float32').reshape(1, -1)
    
    # Optional: Normalize the embedding if your FAISS index expects it
    # faiss.normalize_L2(hypothetical_embedding_np)
    
    # Step C: Use the hypothetical embedding to search the real vector store
    print(f"Searching FAISS with hypothetical embedding for top {k} documents...")
    distances, indices = vectorstore.index.search(hypothetical_embedding_np, k)
    
    retrieved_docs_content = []
    # vectorstore.index_to_docstore_id and vectorstore.docstore._dict are internal
    # properties of LangChain's FAISS class for accessing original documents.
    for i in indices[0]: # indices[0] contains the actual indices
        if i >= 0 and i < len(docs): # Ensure index is valid
            retrieved_docs_content.append(docs[i])
    
    return retrieved_docs_content

In [7]:
# --- 7. Example Usage ---
user_query = "What is RLHF and why is it used?"
top_n_retrieved = 4 # How many real documents we want after HyDE retrieval

retrieved_contexts = hyde_retrieve(user_query, k=top_n_retrieved)

print(f"\n--- Top {top_n_retrieved} Retrieved Documents using HyDE ---")
if retrieved_contexts:
    for i, doc_content in enumerate(retrieved_contexts):
        cleaned_doc_content = doc_content.replace('\n', ' ').strip()
        print(f"Document {i+1}: {cleaned_doc_content[:200]}...") # Print first 200 chars for brevity
else:
    print("No documents retrieved.")


--- HyDE Retrieval for query: 'What is RLHF and why is it used?' ---
Generating hypothetical document...
Hypothetical Document Generated:
---
## RLHF: Reinforcement Learning from Human Feedback

**Definition:**

Reinforcement Learning from Human Feedback (RLHF) is a technique used to train artificial intelligence (AI) models, particularly l...
---
Embedding hypothetical document...
Searching FAISS with hypothetical embedding for top 4 documents...

--- Top 4 Retrieved Documents using HyDE ---
Document 1: Reinforcement Learning from Human Feedback (RLHF) is a technique that fine-tunes language models....
Document 2: It uses human preferences to train a reward model, which then guides the LLM....
Document 3: The primary goal of RLHF is to align AI behavior with human values, making models more helpful and harmless....
Document 4: Step-Back Prompting enhances LLM reasoning by making the model derive high-level concepts first....


Query Rewrite :

In [11]:
# 7. New: Query Rewriting for Multi-Query Retrieval ---

# Define the prompt for generating alternative queries
rewrite_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant that generates multiple search queries based on a single input query. "
     "The goal is to generate alternative phrasings that capture different facets of the original query, "
     "which could lead to broader or more relevant search results. "
     "Return each query on a new line. Only return the queries themselves.\n\n"
     "Example Input: What are the benefits of RLHF?\n"
     "Example Output: "
     "Benefits of RLHF\n"
     "Advantages of Reinforcement Learning from Human Feedback\n"
     "Why use RLHF?\n\n"
     "Generate 3 diverse queries for the following input:"),
    ("user", "{question}")
])
print("\nQuery Rewriting Prompt Template defined.")

# Create the chain for query rewriting
rewrite_chain = rewrite_prompt | llm | StrOutputParser()
print("Query Rewriting Chain defined.")

def multi_query_retrieve(original_query: str, k: int = 3) -> list[str]:
    """
    Performs retrieval using the Multi-Query technique, where an LLM rewrites the original query
    into multiple alternative queries for a broader search.

    Args:
        original_query (str): The user's original query.
        k (int): The number of top relevant real documents to retrieve per rewritten query.

    Returns:
        list[str]: A list of unique retrieved document texts from all rewritten queries.
    """
    print(f"\n--- Multi-Query Retrieval for original query: '{original_query}' ---")

    # Step A: Generate rewritten queries
    print("Generating rewritten queries...")
    rewritten_queries_str = rewrite_chain.invoke({"question": original_query})
    rewritten_queries = [q.strip() for q in rewritten_queries_str.split('\n') if q.strip()]
    
    print(f"Rewritten Queries Generated: {rewritten_queries}")

    all_retrieved_docs_content = set() # Use a set to automatically handle deduplication

    # Step B: Embed and search with each rewritten query
    for i, rewritten_q in enumerate(rewritten_queries):
        print(f"Searching with rewritten query {i+1}: '{rewritten_q}'")
        
        # Embed the rewritten query
        query_embedding = embedding_model.embed_documents([rewritten_q])[0]
        query_embedding_np = np.array(query_embedding).astype('float32').reshape(1, -1)
        
        # Search the FAISS index
        distances, indices = vectorstore.index.search(query_embedding_np, k)
        
        for idx in indices[0]:
            if idx >= 0 and idx < len(docs):
                all_retrieved_docs_content.add(docs[idx]) # Add to set

    return list(all_retrieved_docs_content) # Convert back to list for return



Query Rewriting Prompt Template defined.
Query Rewriting Chain defined.


In [12]:
user_query_rewrite = "Explain the function of FAISS in Gen AI."
top_n_retrieved_rewrite_per_query = 2 # How many docs to retrieve for each rewritten query
retrieved_contexts_rewrite = multi_query_retrieve(user_query_rewrite, k=top_n_retrieved_rewrite_per_query)

print(f"\n--- Retrieved Documents using Multi-Query (Rewriting) ---")
if retrieved_contexts_rewrite:
    for i, doc_content in enumerate(retrieved_contexts_rewrite):
        cleaned_doc_content = doc_content.replace('\n', ' ').strip()
        print(f"Document {i+1}: {cleaned_doc_content[:200]}...")
else:
    print("No documents retrieved using Multi-Query.")


--- Multi-Query Retrieval for original query: 'Explain the function of FAISS in Gen AI.' ---
Generating rewritten queries...
Rewritten Queries Generated: ['FAISS role in generative AI', 'How FAISS is used in Gen AI', 'Function of FAISS for large language models']
Searching with rewritten query 1: 'FAISS role in generative AI'
Searching with rewritten query 2: 'How FAISS is used in Gen AI'
Searching with rewritten query 3: 'Function of FAISS for large language models'

--- Retrieved Documents using Multi-Query (Rewriting) ---
Document 1: Reinforcement Learning from Human Feedback (RLHF) is a technique that fine-tunes language models....
Document 2: FAISS is a library for efficient similarity search and clustering of dense vectors....
Document 3: Step-Back Prompting enhances LLM reasoning by making the model derive high-level concepts first....
Document 4: The primary goal of RLHF is to align AI behavior with human values, making models more helpful and harmless....
